In [ ]:
# default_exp bridge

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#hide
# stellt sicher, dass beim verändern der core library diese wieder neu geladen wird
%load_ext autoreload
%autoreload 2

# Bridge

In [ ]:
#export
from abc import ABC, abstractmethod

from bfh_mt_hs2020_rl_basics.agent import AgentBase

from typing import Iterable, Tuple, List
import numpy as np

from ignite.engine import Engine

from ptan.experience import ExperienceFirstLast

import torch
from torch.optim import Optimizer, Adam
from torch import device

class BridgeBase(ABC):
    
    def __init__(self, agent: AgentBase, optimizer: Optimizer = None, 
                 learning_rate: float = 0.0001, 
                 gamma: float = 0.9, 
                 initial_population: int = 1000, 
                 batch_size: int = 32):
        self.agent = agent
        self.device = agent.device    
        
        self.gamma = gamma
        self.initial_population = initial_population
        self.batch_size = batch_size
                
        if optimizer is not None:
            self.optimzer = optimizer
        else:
            self.optimizer = Adam(self.agent.net.parameters(), lr=learning_rate)
    

    def batch_generator(self):
        self.agent.buffer.populate(self.initial_population)
        while True:
            self.agent.buffer.populate(1)
            yield self.get_sample()


    def _unpack_batch(self, batch: List[ExperienceFirstLast]):
        states, actions, rewards, dones, last_states = [],[],[],[],[]
        
        for exp in batch:
            state = np.array(exp.state)
            states.append(state)
            actions.append(exp.action)
            rewards.append(exp.reward)
            dones.append(exp.last_state is None)
            
            if exp.last_state is None:
                lstate = state  # the result will be masked anyway
            else:
                lstate = np.array(exp.last_state)
            last_states.append(lstate)
            
        return np.array(states, copy=False), \
               np.array(actions), \
               np.array(rewards, dtype=np.float32), \
               np.array(dones,   dtype=np.uint8), \
               np.array(last_states, copy=False)     
    
    
    @abstractmethod
    def get_sample(self, engine: Engine, batchdata):
        pass
    
    @abstractmethod
    def process_batch(self, engine: Engine, batchdata):
        pass
    

## SimpleBridge

In [ ]:
#export
from bfh_mt_hs2020_rl_basics.agent import AgentBase, SimpleAgent

from typing import Iterable, Tuple, List
import numpy as np

from ignite.engine import Engine

from ptan.experience import ExperienceFirstLast

import torch
import torch.nn as nn
from torch.optim import Optimizer, Adam
from torch import device


class SimpleBridge(BridgeBase):
    
    def __init__(self, agent: SimpleAgent, optimizer: Optimizer = None, 
                 learning_rate: float = 0.0001, 
                 gamma: float = 0.9, 
                 initial_population: int = 1000, 
                 batch_size: int = 32):
        
        super(SimpleBridge, self).__init__(agent, optimizer, learning_rate, gamma, initial_population, batch_size)
 

    def get_sample(self):
        return self.agent.buffer.sample(self.batch_size)

    
    def process_batch(self, engine:Engine, batchdata):
        self.optimizer.zero_grad()
       
        loss_v = self._calc_loss(batchdata)

        loss_v.backward()
        self.optimizer.step()
        
        self.agent.iteration_completed(engine.state.iteration)

        return {
            "loss": loss_v.item(),
            "epsilon": self.agent.selector.epsilon,
        }


    def _calc_loss(self, batch: List[ExperienceFirstLast]):
        
        states, actions, rewards, dones, next_states = self._unpack_batch(batch)

        states_v      = torch.tensor(states).to(self.device)
        next_states_v = torch.tensor(next_states).to(self.device)
        actions_v     = torch.tensor(actions).to(self.device)
        rewards_v     = torch.tensor(rewards).to(self.device)
        done_mask     = torch.BoolTensor(dones).to(self.device)

        actions_v         = actions_v.unsqueeze(-1)
        state_action_vals = self.agent.net(states_v).gather(1, actions_v)
        state_action_vals = state_action_vals.squeeze(-1)
        
        with torch.no_grad():
            next_state_vals            = self.agent.tgt_net.target_model(next_states_v).max(1)[0]
            next_state_vals[done_mask] = 0.0
    
        bellman_vals = next_state_vals.detach() * self.gamma + rewards_v
        return nn.MSELoss()(state_action_vals, bellman_vals)

In [ ]:
from bfh_mt_hs2020_rl_basics.agent import SimpleAgent
from bfh_mt_hs2020_rl_basics.env import CarEnv

def basic_simple_init(device=torch.device("cpu")) -> SimpleBridge:
    env = CarEnv()
    agent = SimpleAgent(env, device, gamma=0.9, buffer_size=1000)
    bridge = SimpleBridge(agent, gamma=0.9)
    
    return bridge

In [ ]:
def simple_experiences() -> List[ExperienceFirstLast]:
    return [
        ExperienceFirstLast( np.array([0.0, 0.0, 0.0], dtype=np.float32), np.int64(0), 1.0,  np.array([0.5, 0.5, 0.5], dtype=np.float32)),
        ExperienceFirstLast( np.array([1.0, 1.0, 1.0], dtype=np.float32), np.int64(1), 2.0,  None)        
    ]

In [ ]:
def test_init_cuda():
    assert basic_simple_init(torch.device("cuda")) != None

In [ ]:
def test_init_cpu():
    assert basic_simple_init(torch.device("cpu")) != None

In [ ]:
def test_unpack():
    bridge = basic_simple_init()
    batch = simple_experiences()
    unpacked = bridge._unpack_batch(batch)
    # todo -Checks

In [ ]:
def test_calc_loss():
    bridge = basic_simple_init()
    batch = simple_experiences()
    loss = bridge._calc_loss(batch)
    # todo -Checks  

In [ ]:
from ignite.engine import Engine

def test_process_batch(device=torch.device("cpu")):
    bridge = basic_simple_init(device)
    batch = simple_experiences()
    bridge.process_batch(Engine(bridge.process_batch), batch)
    # todo -Checks

In [ ]:
def test_batch_generator(device=torch.device("cpu")):
    # Test Iterator
    bridge = basic_simple_init(device)
    a = bridge.batch_generator()
    nextbatch = next(a)
    assert len(nextbatch) == 32

In [ ]:
# Basis Tests
test_init_cpu()
test_init_cuda()
test_unpack()
test_calc_loss()
test_process_batch()
test_batch_generator()
test_process_batch(torch.device("cuda"))
test_batch_generator(torch.device("cuda"))

## RainbowBridge

In [ ]:
#export
from bfh_mt_hs2020_rl_basics.agent import AgentBase, RainbowAgent

from typing import Iterable, Tuple, List
import numpy as np

from ignite.engine import Engine

from ptan.experience import ExperienceFirstLast

import torch
import torch.nn as nn
from torch.optim import Optimizer, Adam
from torch import device


class RainbowBridge(BridgeBase):
    
    def __init__(self, agent: RainbowAgent, optimizer: Optimizer = None, 
                 learning_rate: float = 0.0001, 
                 gamma: float = 0.9, 
                 initial_population: int = 1000, 
                 batch_size: int = 32,
                 beta_start: float = 0.4,
                 beta_frames: int = 50000):
        
        super(RainbowBridge, self).__init__(agent, optimizer, learning_rate, gamma, initial_population, batch_size)

        self.beta_start = beta_start
        self.beta = beta_start
        self.beta_frames = beta_frames

    def get_sample(self):
        return self.agent.buffer.sample(self.batch_size, self.beta)
    
    
    def _update_beta(self, idx):
        v = self.beta_start + idx * (1.0 - self.beta_start) / self.beta_frames
        self.beta = min(1.0, v)
        return self.beta
    
    
    def process_batch(self, engine, batch_data):
        
        batch, batch_indices, batch_weights = batch_data
        
        self.optimizer.zero_grad()
        
        loss_v, sample_prios = self._calc_loss(
            batch, 
            batch_weights, 
            gamma=self.gamma**self.agent.steps_count)
        
        loss_v.backward()
        self.optimizer.step()
        
        self.agent.buffer.update_priorities(batch_indices, sample_prios)
        
        self.agent.iteration_completed(engine.state.iteration)
        

        return {
            "loss": loss_v.item(),
            "beta": self._update_beta(engine.state.iteration),
        }
    
    
    def _calc_loss(self, batch, batch_weights, gamma):
        
        states, actions, rewards, dones, next_states = self._unpack_batch(batch)

        states_v        = torch.tensor(states).to(self.device)
        actions_v       = torch.tensor(actions).to(self.device)
        rewards_v       = torch.tensor(rewards).to(self.device)
        done_mask       = torch.BoolTensor(dones).to(self.device)
        batch_weights_v = torch.tensor(batch_weights).to(self.device)

        state_action_values = self.agent.net(states_v).gather(1, actions_v.unsqueeze(-1)).squeeze(-1)
        
        with torch.no_grad():
            next_states_v                = torch.tensor(next_states).to(self.device)
            next_state_values            = self.agent.tgt_net.target_model(next_states_v).max(1)[0]
            next_state_values[done_mask] = 0.0
            expected_state_action_values = next_state_values.detach() * gamma + rewards_v
            
        losses_v = batch_weights_v * (state_action_values - expected_state_action_values) ** 2
        
        return losses_v.mean(), (losses_v + 1e-5).data.cpu().numpy()

In [ ]:
from bfh_mt_hs2020_rl_basics.agent import RainbowAgent
from bfh_mt_hs2020_rl_basics.env import CarEnv

def basic_rainbow_init(device=torch.device("cpu")) -> RainbowBridge:
    env = CarEnv()
    agent = RainbowAgent(env, device, gamma=0.9, buffer_size=1000)
    bridge = RainbowBridge(agent, gamma=0.9)
    
    return bridge

In [ ]:
def test_init_rainbow_cuda():
    assert basic_rainbow_init(torch.device("cuda")) != None

In [ ]:
def test_init_rainbow_cpu():
    assert basic_rainbow_init(torch.device("cpu")) != None

In [ ]:
def test_rainbow_calc_loss():
    bridge = basic_rainbow_init()
    batch = simple_experiences()
    loss = bridge._calc_loss(batch, [0.5,0.5], 0.9)
    # todo -Checks  

In [ ]:
from ignite.engine import Engine

def test_rainbow_process_batch(device=torch.device("cpu")):
    bridge = basic_rainbow_init(device)
    bridge.agent.buffer.buffer = [0,1]
    batch = simple_experiences()
    bridge.process_batch(Engine(bridge.process_batch), (batch, [0,1],[0.5,0.5]))
    # todo -Checks

In [ ]:
def test_rainbow_batch_generator(device=torch.device("cpu")):
    # Test Iterator
    bridge = basic_rainbow_init(device)
    a = bridge.batch_generator()
    nextbatch, idxes, weights = next(a)
    assert len(nextbatch) == 32

In [ ]:
test_rainbow_batch_generator()

In [ ]:
test_init_rainbow_cpu()
test_init_rainbow_cuda()
test_rainbow_calc_loss()
test_rainbow_process_batch()